# Task 6: Outperforming Newton method with Quasi-Newton methods.

In this report, we present the implementation and application of two optimization methods: the Newton method and the BFGS (Broyden–Fletcher–Goldfarb–Shanno) quasi-Newton method. These methods are applied to two test functions: the Rosenbrock function and a custom second function. We will also include detailed logging to trace the execution and compare the performance of these methods in terms of iterations and computation time.


## Optimization Methods
### Newton Method
The Newton method is a second-order optimization technique that uses both the gradient and the Hessian of the function to find the local minima or maxima. It assumes that the function can be locally approximated by a quadratic function, leading to the following iterative update rule:

$$
\mathbf{x}_{k+1} = \mathbf{x}_k - (\nabla^2 f(\mathbf{x}_k))^{-1} \nabla f(\mathbf{x}_k)
$$

where $\mathbf{x}_k$ is the current iterate, $\nabla f(\mathbf{x}_k)$ is the gradient, and $\nabla^2 f(\mathbf{x}_k)$ is the Hessian matrix. The method typically converges very quickly if the initial guess is close to the true solution and the function is well-behaved (e.g., strongly convex).

However, the Newton method has some limitations:
1. **Computation of Hessian**: Calculating the Hessian matrix and its inverse can be computationally expensive, especially for high-dimensional problems.
2. **Requirement for a good initial guess**: The method may fail or converge to a non-optimal point if the initial guess is far from the solution or if the function is not well-behaved.


In [1]:
import numpy as np
import pandas as pd
import time

def detailed_logging(x, grad, hess_inv, iteration, debug=False):
    if debug:
        print(f"Iteration {iteration}: x = {x}, grad = {grad}, hess_inv = {hess_inv}")

def newton_method(f, grad_f, hess_f, x0, tol=1e-8, max_iter=1000, debug=False):
    x = x0
    for i in range(max_iter):
        grad = grad_f(x)
        hess = hess_f(x)
        # Solve the linear system Hx = g instead of computing the inverse
        try:
            step = np.linalg.solve(hess, -grad)
        except np.linalg.LinAlgError:
            if debug:
                print("Hessian is singular or nearly singular at iteration", i)
            break  # Exit if the Hessian is singular or nearly singular

        x = x + step
        detailed_logging(x, grad, np.linalg.inv(hess), i, debug)
        if np.linalg.norm(grad) < tol:
            if debug:
                print(f"Solution found at iteration {i+1}")
            return x, i+1
    return x, max_iter

### BFGS Method
The BFGS method is a quasi-Newton method that builds up an approximation to the inverse Hessian matrix using gradient information only. This method is widely used because it does not require the computation of the Hessian matrix, making it more suitable for large-scale optimization problems. The update rule for the inverse Hessian approximation $\mathbf{B}$ is:

$$
\mathbf{B}_{k+1} = \mathbf{B}_k + \frac{\mathbf{y}_k \mathbf{y}_k^T}{\mathbf{y}_k^T \mathbf{s}_k} - \frac{\mathbf{B}_k \mathbf{s}_k \mathbf{s}_k^T \mathbf{B}_k}{\mathbf{s}_k^T \mathbf{B}_k \mathbf{s}_k}
$$

where $\mathbf{s}_k = \mathbf{x}_{k+1} - \mathbf{x}_k$ and $\mathbf{y}_k = \nabla f(\mathbf{x}_{k+1}) - \nabla f(\mathbf{x}_k)$. The BFGS method iteratively improves the approximation of the inverse Hessian, leading to superlinear convergence.

Key advantages of the BFGS method include:
1. **No need for the Hessian**: It avoids the explicit calculation of the Hessian matrix, making it more computationally efficient for large problems.
2. **Global convergence properties**: It is more robust and often converges more reliably from poor initial guesses compared to the Newton method.

However, the BFGS method also has some limitations:
1. **Storage requirements**: Storing the approximate Hessian matrix can be memory-intensive for very large problems.
2. **Line search dependency**: The efficiency and convergence of the method depend on the effectiveness of the line search algorithm used to determine the step size.


In [2]:
def line_search(f, x, p, grad_f, alpha=1.0, beta=0.5, c=1e-4):
    while f(x + alpha * p) > f(x) + c * alpha * np.dot(grad_f(x), p):
        alpha *= beta  #Reduce step size
    return alpha

def bfgs_method(f, grad_f, x0, tol=1e-6, max_iter=1000, debug=False):
    x = x0
    n = len(x)
    B = np.eye(n)  #initialize B as the identity matrix of dimension n

    for i in range(max_iter):
        grad = grad_f(x)
        if np.linalg.norm(grad) < tol:
            if debug:
                print(f"Solution found at iteration {i+1}")
            return x, i+1

        p = -np.linalg.solve(B, grad)  # Direction to move in
        # Apply line search to find the optimal step size
        alpha = line_search(f, x, p, grad_f)

        x_new = x + alpha * p
        grad_new = grad_f(x_new)
        
        # Update s and y
        s = x_new - x
        y = grad_new - grad
        
        # Update the approximation to the Hessian matrix, B
        if np.dot(y, s) > 0:  # Only update if curvature condition is satisfied
            Bs = B @ s
            B += np.outer(y, y) / np.dot(y, s) - np.outer(Bs, Bs) / np.dot(s, Bs)
        
        x = x_new
        
        if debug:
            detailed_logging(x, grad, np.linalg.inv(B), i)

    return x, max_iter

## Test Functions and Starting Points
### Rosenbrock Function
The Rosenbrock function, also known as the Rosenbrock's valley or Rosenbrock's banana function, is a common test problem for optimization algorithms. It is defined as:

$$
f(\mathbf{x}) = 100 (x_2 - x_1^2)^2 + (1 - x_1)^2
$$

This function has a narrow, curved valley containing the global minimum. The gradient and Hessian of the Rosenbrock function are given by:

$$
\nabla f(\mathbf{x}) = \begin{bmatrix}
-400 x_1 (x_2 - x_1^2) - 2 (1 - x_1) \\
200 (x_2 - x_1^2)
\end{bmatrix}
$$

$$
\nabla^2 f(\mathbf{x}) = \begin{bmatrix}
-400 (x_2 - x_1^2) + 800 x_1^2 + 2 & -400 x_1 \\
-400 x_1 & 200
\end{bmatrix}
$$

Due to its narrow and curved valley, the Rosenbrock function poses a significant challenge for optimization algorithms, making it an excellent candidate for testing the performance and robustness of different methods.


In [3]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

def grad_rosenbrock(x):
    return np.array([
        -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0]),
        200 * (x[1] - x[0]**2)
    ])

def hess_rosenbrock(x):
    return np.array([
        [-400 * (x[1] - x[0]**2) + 800 * x[0]**2 + 2, -400 * x[0]],
        [-400 * x[0], 200]
    ])

### Second Function
The second function used for testing is defined as:

$$
f(\mathbf{x}) = 150 (x_1 x_2)^2 + (0.5 x_1 + 2 x_2 - 2)^2
$$

This function is designed to have multiple minima and different characteristics compared to the Rosenbrock function. The gradient and Hessian of the second function are:

$$
\nabla f(\mathbf{x}) = \begin{bmatrix}
300 x_1 x_2^2 + 0.5 (0.5 x_1 + 2 x_2 - 2) \\
300 x_1^2 x_2 + 2 (0.5 x_1 + 2 x_2 - 2)
\end{bmatrix}
$$

$$
\nabla^2 f(\mathbf{x}) = \begin{bmatrix}
300 x_2^2 + 0.25 & 600 x_1 x_2 + 0.5 \\
600 x_1 x_2 + 0.5 & 300 x_1^2 + 4
\end{bmatrix}
$$

This function provides a complementary challenge to the Rosenbrock function, allowing us to evaluate the performance of the optimization methods on a function with different properties and a different landscape.


In [4]:
def second_function(x):
    return 150 * (x[0] * x[1])**2 + (0.5 * x[0] + 2 * x[1] - 2)**2

def grad_second_function(x):
    return np.array([
        300 * x[0] * x[1]**2 + 0.5 * (0.5 * x[0] + 2 * x[1] - 2),
        300 * x[0]**2 * x[1] + 2 * (0.5 * x[0] + 2 * x[1] - 2)
    ])

def hess_second_function(x):
    return np.array([
        [300 * x[1]**2 + 0.25, 600 * x[0] * x[1] + 0.5],
        [600 * x[0] * x[1] + 0.5, 300 * x[0]**2 + 4]
    ])

### Starting Points

We will test the optimization methods with the following starting points:

In [5]:
start_points_rosenbrock = [
    np.array([1.2, 1.2]), np.array([-1.2, 1]), np.array([0.2, 0.8])
]

start_points_second_function = [
    np.array([-0.2, 1.2]), np.array([3.8, 0.1]), np.array([1.9, 0.6])
]

### Running and Comparing Methods
The following code runs the Newton and BFGS methods on the defined functions with the specified starting points. It logs the results and compares the performance in terms of iterations and time taken.  
The results of the optimization methods are summarized below. Each entry shows the function, method, starting point, found solution, number of iterations, and computation time.

In [6]:
results = []

#collect results for Rosenbrock function
for x0 in start_points_rosenbrock:
    start_time = time.time()
    solution, iterations = newton_method(rosenbrock, grad_rosenbrock, hess_rosenbrock, x0)
    duration = time.time() - start_time
    results.append(('Rosenbrock', 'Newton', x0.tolist(), solution.tolist(), iterations, duration))
    
    start_time = time.time()
    solution, iterations = bfgs_method(rosenbrock, grad_rosenbrock, x0)
    duration = time.time() - start_time
    results.append(('Rosenbrock', 'BFGS', x0.tolist(), solution.tolist(), iterations, duration))

#collect results for the second custom function
for x0 in start_points_second_function:
    start_time = time.time()
    solution, iterations = newton_method(second_function, grad_second_function, hess_second_function, x0)
    duration = time.time() - start_time
    results.append(('Second Function', 'Newton', x0.tolist(), solution.tolist(), iterations, duration))
    
    start_time = time.time()
    solution, iterations = bfgs_method(second_function, grad_second_function, x0)
    duration = time.time() - start_time
    results.append(('Second Function', 'BFGS', x0.tolist(), solution.tolist(), iterations, duration))

#Display results in a Pandas DataFrame
index_labels = [f"Run {i + 1}" for i in range(len(results))]
df_results = pd.DataFrame(results, columns=['Function', 'Method', 'Starting Point', 'Solution', 'Iterations', 'Time (s)'], index=index_labels)
df_results

,Function,Method,Starting Point,Solution,Iterations,Time (s)
Run 1,Rosenbrock,Newton,"[1.2, 1.2]","[1.0, 1.0]",6,0.001997
Run 2,Rosenbrock,BFGS,"[1.2, 1.2]","[1.0000000040561998, 1.0000000091122738]",13,0.001002
Run 3,Rosenbrock,Newton,"[-1.2, 1.0]","[1.0, 1.0]",7,0.000996
Run 4,Rosenbrock,BFGS,"[-1.2, 1.0]","[0.9999999952278598, 0.9999999902393124]",35,0.003001
Run 5,Rosenbrock,Newton,"[0.2, 0.8]","[1.0, 1.0]",6,0.000995
Run 6,Rosenbrock,BFGS,"[0.2, 0.8]","[0.9999999994088168, 0.9999999983055982]",20,0.002001
Run 7,Second Function,Newton,"[-0.2, 1.2]","[0.35283268987680116, 0.0882081724756321]",19,0.002002
Run 8,Second Function,BFGS,"[-0.2, 1.2]","[1.9510723273165978e-09, 1.000000014983516]",17,0.001999
Run 9,Second Function,Newton,"[3.8, 0.1]","[0.35283268990802114, 0.08820817246782824]",12,0.001367
Run 10,Second Function,BFGS,"[3.8, 0.1]","[3.9999999904953767, 3.004652295621832e-11]",8,0.000000


In [7]:
for i in range(0, len(df_results), 2):
    function_newton = df_results.iloc[i]['Function']
    starting_point_newton = df_results.iloc[i]['Starting Point']
    time_newton = df_results.iloc[i]['Time (s)']
    
    function_bfgs = df_results.iloc[i+1]['Function']
    starting_point_bfgs = df_results.iloc[i+1]['Starting Point']
    time_bfgs = df_results.iloc[i+1]['Time (s)']
    
    if time_newton > time_bfgs:
        print(f"Newton is slower than Quasi-Newton for the function {function_newton} and the starting point {starting_point_newton}")
    else:
        print(f"Quasi-Newton is slower than Newton for the function {function_bfgs} and the starting point {starting_point_bfgs}")

Newton is slower than Quasi-Newton for the function Rosenbrock and the starting point [1.2, 1.2]
Quasi-Newton is slower than Newton for the function Rosenbrock and the starting point [-1.2, 1.0]
Quasi-Newton is slower than Newton for the function Rosenbrock and the starting point [0.2, 0.8]
Newton is slower than Quasi-Newton for the function Second Function and the starting point [-0.2, 1.2]
Newton is slower than Quasi-Newton for the function Second Function and the starting point [3.8, 0.1]
Newton is slower than Quasi-Newton for the function Second Function and the starting point [1.9, 0.6]


## Detailed Analysis of Newton vs. BFGS Methods
Our Objective was to identify problems to which the solution of the Newton method exhibits slower runtime performance compared to the BFGS quasi-Newton method when applied to two functions, the Rosenbrock function and a second custom function. The analysis of runtime is critical as it reflects the efficiency of the algorithm in practical scenarios, especially in large-scale applications where runtime can significantly impact overall performance.

### Experiment Setup
- **Functions**: This method leverages the full Hessian matrix to compute updates, offering precise convergence in ideal scenarios but at the cost of increased computational overhead.
- **Methods**: As a quasi-Newton method, BFGS approximates the Hessian matrix updates, which can significantly reduce computational demands, particularly beneficial in large-dimensional problems or under conditions where the Hessian is difficult to compute.
- **Starting Points**: Multiple starting points were chosen to assess the robustness of the algorithms under varying initial conditions.

We added a special technique called a "line search algorithm" to the BFGS optimization method. This technique adjusts the step size 𝛼, each time the method tries to find the minimum of a function. The step size is how far the method moves along in its search for the lowest point.  
The purpose of this line search is to find the most effective step size for each move. This ensures that each step improves the position towards finding the function's minimum value without going too far or not far enough. It's like fine-tuning each step to make sure it's just right.  
We used a straightforward approach known as the "backtracking line search," which is based on a simple optimization rule called the Armijo condition. This starts with a guess for the step size and keeps reducing it gradually until the improvement in the function's value is good enough to meet a specific requirement. This requirement involves checking the slope (directional derivative) at the current point and adjusting the step size accordingly.  
By carefully adjusting the step size on each iteration, this line search helps the BFGS method to avoid two main problems: overshooting (going too far and missing the minimum) and taking steps that are too small, which can slow down the process. As a result, this adjustment helps the method find the minimum point of the function more quickly and reliably.  

This enhancement makes the BFGS method more efficient, especially in tricky optimization problems where finding the right step size isn't straightforward. It's like having a smart assistant that helps the method decide how big each step should be to efficiently reach the goal.  

### Results
- **Rosenbrock Function**: For the first starting point ([1.2, 1.2]), the Newton's method is performing slower than Quasi-Newton's method. For all the other starting points ([-1.2, 1.0]), and ([0.2, 0.8]), the Newton method was consistently faster in terms of runtime compared to BFGS. This might be attributed to the second-order nature of the Newton method, which, despite the overhead of computing the Hessian, converges in fewer iterations when close to the solution within a well-structured problem like the Rosenbrock function.

- **Second Function**: For starting points ([-0.2, 1.2]), ([3.8, 0.1]), and ([1.9, 0.6]), the Newton method is relatively slower than Quas-Newton method. This suggests that in scenarios where the function landscape is complex or ill-conditioned with respect to the initial point, the BFGS method's less computationally intensive update rule can achieve faster convergence in runtime.

### Interpretation

- **Efficiency**: The results clearly indicate that while the Newton method can be highly efficient in terms of iterations due to its use of exact second-order information, its runtime efficiency is highly dependent on the nature of the function and the proximity of the initial point to the solution. The BFGS method, with its approximative approach, offers a significant advantage in scenarios where the Hessian computation is expensive or the function is poorly conditioned.

- **Application**: These findings are crucial for applications where runtime is a critical factor. For functions similar to our second test function or in high-dimensional optimization problems, BFGS may be preferred over Newton.

### Conclusion
The experiment successfully identified specific cases where the BFGS quasi-Newton method outperforms the Newton method in terms of runtime.